# Example of implementation: an extending ice cube

## Intro, imports, and images
This notebook applies the finite element method to obtain the velocity field of a cube of ice, of side length $h$, spreading horizontally and compressing vertically under its own weight. The setup is depicted in Figure 1 of the accompanying manuscript (Miele et al., in review), which is reproduced [below](#Figure-1). This script uses the symbolic math library ```SymPy``` to solve the FEM equations symbolically, and it provides the option of solving numerically with functions from ```NumPy``` and ```SciPy```. The workflow of the code mirrors the workflow beginning on page 15 of the accompanying manuscript and can be used to verify the derivations presented in that section. All equation numbers are in reference to the accompanying manuscript (insert hyperlink once this is up on a preprint server). Access this notebook interactively with Binder: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/chrismiele/ice_cube/main)

In [1]:
import sympy as sp
import numpy as np
from scipy.integrate import dblquad, tplquad #double and triple numeric integration
import time
start = time.time()

This code will be slightly more flexible than the example presented in the main text. The default settings will solve the problem posed in the manuscript: the hydrostatic approximation for a cube in **biaxial extension** (i.e., extending both longitudinally and laterally, with ice cliffs at $x = h$ and $y = h$). This script will provide the additional option of solving for **uniaxial extension** (i.e., extending only longitudinally, between slippery lateral barriers at $y = h$ and $y = 0$). Setting ```biaxial_spread``` to ```False``` in the code block below will solve the uniaxial extension problem.

Additionally, either of these problems can be solved via the **hydrostatic approximation**, as done in the manuscript, or via the **Blatter-Pattyn** (BP) approximation. Solve BP by setting ```hydrostatic_approximation``` to ```False```.

Finally, by default, this notebook provides **symbolic** solutions (in which all variables are kept general). However, if ```symbolic_solution``` is set to ```False```, the equations are solved **numerically**, given values specified in the [third cell block](#Spatial-variables,-dimensions,-and-physical-parameters). Solve symbolically to conceptually explore each step in the algorithm, or solve numerically to utilize more efficient calculation techniques.

To compare the efficiency of these various solution approaches, uncomment the final line of code at the [end of this notebook](#Nonlinear-interpolated-solution) and run the entire script.

In [2]:
biaxial_spread = True 
hydrostatic_approximation = True 
symbolic_solution = True

### Figure 1

The domain over which we implement our finite element system of equations. The shaded area represents the quadrant of the ice block within which we obtain our 3D velocity field. The block is meshed as indicated by the node numbers shown, with node 1 at the origin, node 2 at $(0, h, 0)$, and so on. The bottom surface ($z = 0$) rests on a hard, slippery barrier.

<img src='cube.PNG' width='500'>

## Spatial variables, dimensions, and physical parameters

Define the spatial coordinates $x$, $y$, and $z$, alongside the side length $h$, viscosity $\mu$, density $\rho$, gravitational constant $g$, rate factor $A$, and flow exponent $n$. Units are meters, seconds, kilograms, Pascals, etc. 

In [3]:
x, y, z = sp.symbols('x, y, z', real=True)

if symbolic_solution: #if we want a symbolic solution:
    h, μ, ρ, g, A, n = sp.symbols('h, μ, ρ, g, A, n', positive=True, constant=True) #keep all variables unspecified
else: #if solving numerically:
    h = 100 #side length in meters
    μ, ρ, g, A, n = 4e13, 917, 9.8, 1e-23, 3


As well, define the lithostatic vector $\boldsymbol{\Lambda}$ and body force vector $\mathbf{b}$. For the hydrostatic approximation, these terms are given in Table 1 from Miele et al. (in review) as

$$\boldsymbol{\Lambda} := \rho g(h-z)\begin{bmatrix}1 & 1 & 1 & 0 & 0 & 0\end{bmatrix}^T \hspace{1cm} \text{and} \hspace{1cm}\mathbf{b} := -\rho g\begin{bmatrix}0 & 0 & 1\end{bmatrix}^T$$ 

However, if solving Blatter-Pattyn, the dimension of the problem is reduced (refer again to Table 1), and the definitions of $\boldsymbol{\Lambda}$ and $\mathbf{b}$ become

$$\boldsymbol{\Lambda} := \rho g(h-z)\begin{bmatrix}1 & 1 & 0 & 0 & 0\end{bmatrix}^T \hspace{1cm} \text{and} \hspace{1cm}\mathbf{b} := -\rho g\begin{bmatrix}0 & 0\end{bmatrix}^T$$

Notice that the Blatter-Pattyn definitions are nearly identical to the hydrostatic definitions, but with the third components omitted. Below, we provide the appropriate descriptions of $\boldsymbol{\Lambda}$ and $\mathbf{b}$ for whichever approximation has been selected. We'll also include some other variables to help with bookkeeping:

In [4]:
#start with the most general descriptions of Λ and b:
Λ = ρ*g*(h-z)*sp.Matrix((1, 1, 1, 0, 0, 0))
b = -ρ*g*sp.Matrix((0, 0, 1))

#general list of nodal velocity components:
nodal_velocity = ['u_1x', 'u_1y', 'u_1z', 'u_2x', 'u_2y', 'u_2z', 'u_3x', 'u_3y', 'u_3z', 'u_4x', 'u_4y', 'u_4z',
               'u_5x', 'u_5y', 'u_5z', 'u_6x', 'u_6y', 'u_6z', 'u_7x', 'u_7y', 'u_7z', 'u_8x', 'u_8y', 'u_8z']
#general list of velocity field components:
velocity = ['u_x', 'u_y', 'u_z']

#reduce the dimension of these variables, if necessary:
if not hydrostatic_approximation: #if solving BP:
    Λ.row_del(2) #remove the third row from Λ
    b.row_del(2) #and from b
    del nodal_velocity[2::3] #remove each nodal u_iz component
    del velocity[2] #and remove u_z from the velocity field description

dim = len(velocity) #dimension of the velocity solution: 3 for hydrostatic, 2 for BP
n_nodes = 8 #one node at each corner of the cube 

print('This notebook is now set up to solve for', velocity, 'which has nodal components:')
sp.Matrix(nodal_velocity).T

This notebook is now set up to solve for ['u_x', 'u_y', 'u_z'] which has nodal components:


Matrix([[u_1x, u_1y, u_1z, u_2x, u_2y, u_2z, u_3x, u_3y, u_3z, u_4x, u_4y, u_4z, u_5x, u_5y, u_5z, u_6x, u_6y, u_6z, u_7x, u_7y, u_7z, u_8x, u_8y, u_8z]])

## Symmetric and resistive gradient operators 

For the hydrostatic approximation, the symmetric and resistive gradient operators are defined in Table 1 from the manuscript as

$$\boldsymbol{\nabla}_S := \begin{bmatrix}\frac{\partial}{\partial x} & 0 & 0\\
0 & \frac{\partial}{\partial y} & 0\\
0 & 0 & \frac{\partial}{\partial z}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x} & 0\\
\frac{\partial}{\partial z} & 0 & \frac{\partial}{\partial x}\\
0 & \frac{\partial}{\partial z} & \frac{\partial}{\partial y}
\end{bmatrix} \hspace{1cm} \text{and} \hspace {1cm}\boldsymbol{\nabla}_R := \begin{bmatrix}4\frac{\partial}{\partial x} & 2\frac{\partial}{\partial y} & 0\\
2\frac{\partial}{\partial x} & 4\frac{\partial}{\partial y} & 0\\
2\frac{\partial}{\partial x} & 2\frac{\partial}{\partial y} & 2\frac{\partial}{\partial z}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x} & 0\\
\frac{\partial}{\partial z} & 0 & \frac{\partial}{\partial x}\\
0 & \frac{\partial}{\partial z} & \frac{\partial}{\partial y}
\end{bmatrix}$$

For BP, Table 1 shows that the appropriate definitions are similar to those given above, but with the third row and column omitted from each. For either case, we include these operators as functions, ```nabla_S``` and ```nabla_R```. For example, the function ```nabla_S``` will take a matrix ```M``` as input and return the matrix product $\boldsymbol{\nabla}_S\mathbf{M}$. 

In [5]:
def nabla_S(M):
    out = sp.zeros(dim+3, M.cols) #initialize output
    for i in range(M.cols):
        out[0, i] = sp.diff(M[0, i], x) #∂/∂x(M[0, i])
        out[1, i] = sp.diff(M[1, i], y) #∂/∂y(M[1, i])
        out[-3, i] = sp.diff(M[0, i], y) + sp.diff(M[1, i], x)
        out[-2, i] = sp.diff(M[0, i], z) 
        out[-1, i] = sp.diff(M[1, i], z)      
        if hydrostatic_approximation: #include the 3rd row and column:
            out[2, i] = sp.diff(M[2, i], z) #etc.
            out[-2, i] += sp.diff(M[2, i], x)
            out[-1, i] += sp.diff(M[2, i], y)
    return out #return the dot product ∇_S*M

def nabla_R(M):
    out = sp.zeros(dim+3, M.cols) #initialize output
    for i in range(M.cols):
        out[0, i] = 4*sp.diff(M[0, i], x) + 2*sp.diff(M[1, i], y) #4∂/∂x(M[0, i]) + 2∂/∂y(M[1, i])
        out[1, i] = 2*sp.diff(M[0, i], x) + 4*sp.diff(M[1, i], y) #etc.
        out[-3, i] = sp.diff(M[0, i], y) + sp.diff(M[1, i], x)
        out[-2, i] = sp.diff(M[0, i], z)
        out[-1, i] = sp.diff(M[1, i], z) 
        if hydrostatic_approximation: #include the 3rd row and column:
            out[2, i] = 2*sp.diff(M[0, i], x) + 2*sp.diff(M[1, i], y) + 2*sp.diff(M[2, i], z)
            out[-2, i] += sp.diff(M[2, i], x)
            out[-3, i] += sp.diff(M[2, i], y)
    return out #return the dot product ∇_R*M

## Shape functions and their gradients

The nodal shape functions $N_1$ through $N_8$ are defined such that $N_i$ evaluates to one at node $i$ and zero at every other node. It can be directly verified that the functions below satisfy this property for the setup shown in [Figure 1](#Figure-1). These functions are also given in Equation 26 of Miele et al. (in review).

In [6]:
N1 = (h-x)*(h-y)*(h-z)/h**3
N2 = (h-x)*y*(h-z)/h**3
N3 = x*y*(h-z)/h**3
N4 = x*(h-y)*(h-z)/h**3
N5 = x*(h-y)*z/h**3
N6 = (h-x)*(h-y)*z/h**3
N7 = (h-x)*y*z/h**3
N8 = x*y*z/h**3

#For example, node 2 is at (0, h, 0) and node 5 is at (h, 0, h)
print('N2 evaluated at node 2:', 'N2(0, h, 0) =', N2.subs({x:0, y:h, z:0}))
print('N2 evaluated at node 5:', 'N2(h, 0, h) =', N2.subs({x:h, y:0, z:h}))

N2 evaluated at node 2: N2(0, h, 0) = 1
N2 evaluated at node 5: N2(h, 0, h) = 0


Next, it's useful to express these shape function in matrix form. For the hydrostatic approximation, the matrix has the form 

$$\mathbf{N} := \begin{bmatrix}N_1 & 0 & 0 & N_2 & 0 & 0 & ... & N_8 & 0 & 0\\
0 & N_1 & 0 & 0 & N_2 & 0 & ... & 0 & N_8 & 0\\
0 & 0 & N_1 & 0 & 0 & N_2 & ... & 0 & 0 & N_8
\end{bmatrix}$$

If solving Blatter-Pattyn, $\mathbf{N}$ has reduced dimension, with

$$\mathbf{N} := \begin{bmatrix}N_1 & 0 & N_2 & 0 & ... & N_8 & 0\\
0 & N_1 & 0 & N_2 & ... & 0 & N_8\\
\end{bmatrix}$$

In [7]:
#first, write the shape functions as an 8-by-1 vector
vec_N = sp.Array((N1, N2, N3, N4, N5, N6, N7, N8))

#now construct the shape function N as a dim-by-dim*n_nodes matrix
N = sp.zeros(dim, dim*n_nodes) #initialize the matrix N
I = sp.eye(dim) #dim-by-dim identity matrix
for i in range(n_nodes):
    N[0:dim, dim*i:dim*i+dim] = I*vec_N[i] #each ith square submatrix gets assigned I*N_i

#output a visual representation of N:
print('Shape function in matrix form:')
N

Shape function in matrix form:


Matrix([
[(h - x)*(h - y)*(h - z)/h**3,                            0,                            0, y*(h - x)*(h - z)/h**3,                      0,                      0, x*y*(h - z)/h**3,                0,                0, x*(h - y)*(h - z)/h**3,                      0,                      0, x*z*(h - y)/h**3,                0,                0, z*(h - x)*(h - y)/h**3,                      0,                      0, y*z*(h - x)/h**3,                0,                0, x*y*z/h**3,          0,          0],
[                           0, (h - x)*(h - y)*(h - z)/h**3,                            0,                      0, y*(h - x)*(h - z)/h**3,                      0,                0, x*y*(h - z)/h**3,                0,                      0, x*(h - y)*(h - z)/h**3,                      0,                0, x*z*(h - y)/h**3,                0,                      0, z*(h - x)*(h - y)/h**3,                      0,                0, y*z*(h - x)/h**3,                0,          0, x*y*

The gradients of the shape function are $\mathbf{B}_S = \boldsymbol{\nabla}_S\mathbf{N}$ and $\mathbf{B}_R = \boldsymbol{\nabla}_R\mathbf{N}$, which are evaluated as

In [8]:
B_S = nabla_S(N)
B_R = nabla_R(N)

print('Resistive gradient of the shape function matrix, ∇_R*N := B_R')
B_R #compare with Equation 28 of the manuscript for the biaxial extension setup

Resistive gradient of the shape function matrix, ∇_R*N := B_R


Matrix([
[-4*(h - y)*(h - z)/h**3, -2*(h - x)*(h - z)/h**3,                       0,    -4*y*(h - z)/h**3, 2*(h - x)*(h - z)/h**3,                    0, 4*y*(h - z)/h**3, 2*x*(h - z)/h**3,              0, 4*(h - y)*(h - z)/h**3,    -2*x*(h - z)/h**3,                    0, 4*z*(h - y)/h**3,    -2*x*z/h**3,                0,    -4*z*(h - y)/h**3,    -2*z*(h - x)/h**3,                      0,    -4*y*z/h**3, 2*z*(h - x)/h**3,                0, 4*y*z/h**3, 2*x*z/h**3,          0],
[-2*(h - y)*(h - z)/h**3, -4*(h - x)*(h - z)/h**3,                       0,    -2*y*(h - z)/h**3, 4*(h - x)*(h - z)/h**3,                    0, 2*y*(h - z)/h**3, 4*x*(h - z)/h**3,              0, 2*(h - y)*(h - z)/h**3,    -4*x*(h - z)/h**3,                    0, 2*z*(h - y)/h**3,    -4*x*z/h**3,                0,    -2*z*(h - y)/h**3,    -4*z*(h - x)/h**3,                      0,    -2*y*z/h**3, 4*z*(h - x)/h**3,                0, 2*y*z/h**3, 4*x*z/h**3,          0],
[-2*(h - y)*(h - z)/h**3, -2*(h - x)*(h - z)/

## Boundary conditions

### Velocity (Dirichlet) boundary conditions

Nodal velocities are written symbolically in the previously-defined list ```nodal_velocity```. Many of these components are already known by nature of the setup (see the paragraph following Equation 29 in the paper), and, thus, constitute **Dirichlet boundary conditions**. For example, in the biaxial, hydrostatic case, $u_{1z} = u_{2z} = u_{3z} = u_{4z} = 0$ because the bottom of the cube rests on a hard barrier. By symmetry, $u_x = 0$ on the $y$ axis and $u_y = 0$ on the $x$ axis, and so $u_{1x} = u_{2x} = u_{6x} = u_{7x} = u_{1y} = u_{4y} = u_{5y} = u_{6y} = 0$. For the uniaxial case (i.e., ```biaxial_spread == False```), there is no lateral flow through the sides, and so each $y$ velocity component is also zero. If solving the Blatter-Pattyn approximation (i.e., ```hydrostatic_approximation == False```), then there are no boundary conditions on $u_z$ since this component is not solved for. 

To code this information, we define a list, ```BCs```, having the same length as ```nodal_velocity``` but consisting only of 1s and 0s. ```BCs``` will be 0 wherever the corresponding nodal velocity is prescribed to be zero, and 1 wherever velocity is unknown. Beginning with the most general case of the hydrostatic approximation in biaxial extension, we have:

In [9]:
#identify the components with Dirichlet BCs for the biaxial, hydrostatic case:
BCs = [0, 0, 0, #u_1x, u_1y, u_1z = 0
     0, 1, 0, #u_2x, u_2z = 0 while u_2y is unknown
     1, 1, 0, #u_3x, u_3y are unknown while u_3z is zero
     1, 0, 0, #etc.
     1, 0, 1, 
     0, 0, 1, 
     0, 1, 1,
     1, 1, 1]

if not biaxial_spread: #if solving for uniaxial extension:
    for i in range(len(BCs)):
        if i%3 == 1: #for u_1y, u_2y, u_3y, etc.:
            BCs[i] = 0 #set this component to zero

if not hydrostatic_approximation: #if solving Blatter-Pattyn:
    del BCs[2::3] #delete u_1z, u_2z, u_3z, etc.
   
print('Zeros indicate the velocity components associated with Dirichlet boundary conditions.')
print('Components are ordered', nodal_velocity[:4], '...', nodal_velocity[-4:], ':')
sp.Matrix(BCs).T

Zeros indicate the velocity components associated with Dirichlet boundary conditions.
Components are ordered ['u_1x', 'u_1y', 'u_1z', 'u_2x'] ... ['u_7z', 'u_8x', 'u_8y', 'u_8z'] :


Matrix([[0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1]])

### Traction (Neumann) boundary conditions

In the biaxial spreading case discussed in the manuscript, the positive $x$ and $y$ faces are associated with depth-averaged ice cliff boundary conditions:

$$\mathbf{t}_X = \rho g\left(z - \frac{h}{2}\right)\hat{\mathbf{i}}\hspace{1cm} \text{and} \hspace{1cm} \mathbf{t}_Y = \rho g\left(z - \frac{h}{2}\right)\hat{\mathbf{j}}$$

For consistency with the uniaxial case, we can also include a placeholder boundary condition at the negative $Y$ face, setting $\mathbf{t}_{negY} = \mathbf{0}$.

In [10]:
i_hat = I.col(0) #x basis vector is the 1st column of the identity matrix
j_hat = I.col(1) #y basis vector is the 2nd column

t_X = ρ*g*(z - h/2)*i_hat #t_X = σ_xx|_{x=h}*i
t_Y = ρ*g*(z - h/2)*j_hat #t_Y = σ_yy|_{y=h}*j
t_negY = 0*-j_hat #placeholder value t_negY = zero vector

In the uniaxial case, the positive $x$ boundary remains a free ice cliff, but the positive and negative $y$ boundaries must have zero lateral extension; that is, $\tau_{yy}|_{y = h} = \tau_{yy}|_{y = 0} = 0$. This is the case when the net stress, $\sigma_{yy}$, is simply equal to the overburden pressure at these locations. Therefore, in a uniaxial extending regime,

$$\mathbf{t}_{Y} = \rho g(z-h)\hat{\mathbf{j}}\hspace{1cm} \text{and} \hspace{1cm}\mathbf{t}_{negY} = \rho g(z-h)\left(-\hat{\mathbf{j}}\right)$$

In [11]:
if not biaxial_spread:
    t_Y = ρ*g*(z - h)*j_hat #t_Y = σ_yy|_{y=h}*j
    t_negY = ρ*g*(z - h)*-j_hat #t_negY = σ_yy|_{y=0}*-j

## Stiffness matrix and force vector calculation

### Stiffness matrix $\mathbf{K}$

We evaluate the stiffness matrix $\mathbf{K}$ as the triple integral $\iiint_{x,y,z = 0}^h\mathbf{B}_S^T\mu\mathbf{B}_Rdxdydz$ (see Eq. 33). The symbolic solution will be calculated with SymPy's ```integrate``` function, and the numerical solution will be calculated using SciPy's ```tplquad``` function. 

Since ```sympy.integrate``` is somewhat slow (it takes > 20 seconds to compute $24^2$ triple integrals symbolically), we'll calculate only the entries of $\mathbf{K}$ which are needed. Because of how $\mathbf{K}$ will eventually be [partitioned](#Swapping-and-partitioning), any entry of $\mathbf{K}$ corresponding to a Dirichlet boundary condition will not influence the solution step; these entries can be artificially set to zero (this cuts the calculation time down by about 75\%, but it won't make a noticeable difference if solving numerically with ```scipi.integrate.tplquad```, which is already more efficient).  

In [12]:
K = sp.zeros(dim*n_nodes, dim*n_nodes) #initialize K as a square matrix of zeros
integrand_K = B_S.T*μ*B_R #we'll populate K by integrating select entries of this square matrix

for i in range(K.rows): #for each row:
    for j in range(K.cols): #and each column:
        if BCs[i] != 0 and BCs[j] != 0: #if NEITHER the row nor the column correspond to a velocity BC:
            integrand = integrand_K[i, j] #we'll integrate this entry using one of the methods below.
            if symbolic_solution: #if we want a symbolic solution, we'll integrate symbolically:
                K[i, j] =  sp.integrate(integrand, (x, 0, h), (y, 0, h), (z, 0, h))
            else: #otherwise, if we just want a numerical solution:
                integrand = sp.lambdify((x, y, z), integrand) #interpret integrand as a function
                K[i, j] = tplquad(integrand, 0, h, 0, h, 0, h, epsabs=1e-10*μ)[0] #and integrate numerically
        #elif the entry DOES correspond to a velocity BC:
            #leave it as zero.

#Note: setting tplquad argument espabs=1e-10*μ helps avoid a roundoff-error warning.
#espabs sets the absolute error tolerance; default value is on the order of 1e-8.
#Because μ is such a large number, roundoff errors are inflated to > unit scale, producing the warning.
#However, these values are negligible compared to the entries of K, which are of scale μ. 
#Avoid the warning by increasing the error tolerance to 10 orders of magnitude smaller than μ. 

### Force vector $\mathbf{f}$

The force vector can be seperated into components $\mathbf{f} = \mathbf{f}_\Gamma + \mathbf{f}_\Omega + \mathbf{f}_\Lambda$, each of which is calculated similarly to $\mathbf{K}$ (see Eq. 16 for the general theory, and Eq. 34 for application to the biaxial extension problem).

In [13]:
#initialize each component of f:
f_Γx, f_Γy, f_Γnegy = sp.zeros(dim*n_nodes, 1), sp.zeros(dim*n_nodes, 1), sp.zeros(dim*n_nodes, 1)
f_Ω, f_Λ = sp.zeros(dim*n_nodes, 1), sp.zeros(dim*n_nodes, 1)
    
integrand_Γx = N.T.subs(x, h)*t_X #integrand_Γx will be integrated in calculating f_Γx
integrand_Γy = N.T.subs(y, h)*t_Y #for calculating f_Γy
integrand_Γnegy = N.T.subs(y, 0)*t_negY #for calculating f_Γnegy
integrand_Ω = N.T*b #for calculating f_Ω
integrand_Λ = B_S.T*Λ #for calculating f_Λ

for i in range(dim*n_nodes):
    if symbolic_solution: #use sympy's symbolic integration function:
        f_Γx[i] = sp.integrate(integrand_Γx[i], (y, 0, h), (z, 0, h)) #term 1 of Eq. 34
        f_Γy[i] = sp.integrate(integrand_Γy[i], (x, 0, h), (z, 0, h)) #term 2 of Eq. 34
        f_Γnegy[i] = sp.integrate(integrand_Γnegy[i], (x, 0, h), (z, 0, h)) #nonzero only if uniaxial
        f_Ω[i] = sp.integrate(integrand_Ω[i], (x, 0, h), (y, 0, h), (z, 0, h)) #term 3
        f_Λ[i] = sp.integrate(integrand_Λ[i], (x, 0, h), (y, 0, h), (z, 0, h)) #term 4
    else: #use scipy's numeric integration functions:
        f_Γx[i] = dblquad(sp.lambdify((y, z), integrand_Γx[i]), 0, h, 0, h)[0] #term 1 of Eq. 34
        f_Γy[i] = dblquad(sp.lambdify((x, z), integrand_Γy[i]), 0, h, 0, h)[0] #term 2
        f_Γnegy[i] = dblquad(sp.lambdify((x, z), integrand_Γnegy[i]), 0, h, 0, h)[0] #nonzero only if uniaxial
        f_Ω[i] = tplquad(sp.lambdify((x, y, z), integrand_Ω[i]), 0, h, 0, h, 0, h)[0] #term 3
        f_Λ[i] = tplquad(sp.lambdify((x, y, z), integrand_Λ[i]), 0, h, 0, h, 0, h)[0] #term 4
        
f_Γ = f_Γx + f_Γy + f_Γnegy #sum the components of f_Γ
f = f_Γ + f_Ω + f_Λ #and obtain the total force vector 

## Swapping and partitioning

In order to partition the finite element system of equations into the form shown in Equation 17 of the manuscript, it is necessary to systematically sort stiffness matrix $\mathbf{K}$, nodal velocity vector $\mathbf{d}$, and force vector $\mathbf{f}$. The goal is to rearrange $\mathbf{d}$ so that the velocity components chosen as boundary conditions appear as the top entries, with the unknown velocity components at the bottom $-$ while simultaneously sorting $\mathbf{K}$ and $\mathbf{f}$ so as to preserve the system of linear equations represented by $\mathbf{K}\mathbf{d} = \mathbf{f}$. 

Bookkeeping will be done via a list, ```order```, which will document the new placement of each component of $\mathbf{d}$ after sorting.  

In [26]:
order = sorted(range(dim*n_nodes), key = lambda i : BCs[i]) 
print('This list shows the new placement of the ith velocity component after sorting:')
sp.Matrix(order).T

This list shows the new placement of the ith velocity component after sorting:


Matrix([[0, 1, 2, 3, 5, 8, 10, 11, 13, 15, 16, 18, 4, 6, 7, 9, 12, 14, 17, 19, 20, 21, 22, 23]])

For example, velocity component ```0``` (i.e., the first entry from the list ```nodal_velocity```) has remained in the top position, while velocity component ```6``` has been moved further down. (Compare this new arrangement with the [Dirichlet boundary conditions](#Velocity-(Dirichlet)-boundary-conditions) chosen earlier.)

Next, we'll use this bookkeeper list to help rearrange the elements of $\mathbf{K}$ and $\mathbf{f}$. 

In [16]:
#initialize the element-swapped versions of f and K
K_swapped = sp.zeros(K.rows, K.cols)
f_swapped = sp.zeros(f.rows, f.cols)

#fill the matrices with their appropriate values:
for i in range(K.rows):
    f_swapped[i] = f[order[i]] #swap the correct rows of f
    for j in range(K.cols):
        K_swapped[i, j] = K[order[i], order[j]] 
        #and swap the correct rows AND columns of K

#because of how we populated K, all nonzero entries now appear in the bottom right submatrix of K_swapped
#uncomment the next line to see the full matrix K_swapped:
#K_swapped

With reference to Equation 17 from Miele et al. (in review), we now compute $\mathbf{K}_F$ and $\mathbf{f}_F$:

In [17]:
#to partition, first count the number of velocity BCs
VBCs = BCs.count(0) #This is the number of 0s in the previously-defined list, BCs

#Kf and ff are shown in Equation 17
ff = f_swapped[VBCs:, :] #the bottom components of f_swapped
Kf = K_swapped[VBCs:, VBCs:] #the bottom right components of K_swapped

## Computing the linear rheology solution

### Linear nodal solution

The nodal solution is obtained via Equation 18 of the paper, with the added simplification that $\mathbf{d}_E$ is just the zero vector (because every velocity boundary condition is zero). The explicit solution for $\mathbf{d}_F$ is $\mathbf{d}_F = \mathbf{K}_F^{-1}\mathbf{f}_F$. However, in general, it is more efficient to solve systems of linear equations without performing matrix inversions. Instead, use SymPy's ```linsolve``` function (if solving symbolically) or NumPy's ```linalg.solve``` function (if solving numerically):

In [18]:
if symbolic_solution: #if solving symbolically:
    df = sp.linsolve([Kf, ff]) #solves the system Kf*X = ff symbolically for unknown X
    df = list(list(df)[0]) #write the solution in list form
else:
    Kf, ff = np.array(Kf, 'float'), np.array(ff, 'float') #interpret Kf, ff as float-valued numpy arrays
    df = np.linalg.solve(Kf, ff) #solves the same system numerically
    df = [float(df[i]) for i in range(len(df))] #write the solution in list form

The complete (but still [sorted](#Swapping-and-partitioning)!) list of nodal velocities is now given as the vector $\begin{bmatrix} \mathbf{d}_E & \mathbf{d}_F \end{bmatrix}^T$:

In [19]:
de = [0]*VBCs #the list of known (i.e., zero) velocities
d_swapped = de + df #the complete list of nodal velocities, still sorted

Put the elements of ```d_sorted``` back into their original places:

In [20]:
d = [0]*dim*n_nodes #initialize the final nodal solution
for i in range(len(d_swapped)): #for each entry in d_swapped:
    d[order[i]] = d_swapped[i] #place that entry into its correct position in d
d = sp.Matrix(d) #d is now in the original order

print('Nodal velocity solution (linear rheology)')
print('Components are in their original order,', nodal_velocity[:4], '...', nodal_velocity[-4:], ':')
d.T

Nodal velocity solution (linear rheology)
Components are in their original order, ['u_1x', 'u_1y', 'u_1z', 'u_2x'] ... ['u_7z', 'u_8x', 'u_8y', 'u_8z'] :


Matrix([[0, 0, 0, 0, g*h**2*ρ/(12*μ), 0, g*h**2*ρ/(12*μ), g*h**2*ρ/(12*μ), 0, g*h**2*ρ/(12*μ), 0, 0, g*h**2*ρ/(12*μ), 0, -g*h**2*ρ/(6*μ), 0, 0, -g*h**2*ρ/(6*μ), 0, g*h**2*ρ/(12*μ), -g*h**2*ρ/(6*μ), g*h**2*ρ/(12*μ), g*h**2*ρ/(12*μ), -g*h**2*ρ/(6*μ)]])

### Linear interpolated solution

The nodal solution can be used to obtain the *continuous* velocity solution, $\mathbf{u}$, by interpolation. This is done, by approximation, using the element shape function matrix, with $\mathbf{u} \approx \mathbf{N}\mathbf{d}$.

In [21]:
u = N*d #interpolate the nodal solution
u = sp.simplify(u) #group like terms to express compactly

if not symbolic_solution: #if solving numerically:
    def round_expr(expr): #eliminate rounding-error-sized coefficients < 1e-15
        return expr.xreplace({n : round(n, 15) for n in expr.atoms(sp.Number)})
    u = round_expr(u)

print('Interpolated velocity solution (linear rheology) for', velocity, ':')
u #compare with the biaxial extension solution of Equation 38 from the paper

Interpolated velocity solution (linear rheology) for ['u_x', 'u_y', 'u_z'] :


Matrix([
[g*h*x*ρ/(12*μ)],
[g*h*y*ρ/(12*μ)],
[-g*h*z*ρ/(6*μ)]])

## Postprocessing

The next goal will be to compute the [nonlinear rheology solution](#Nonlinear-rheology-solution). However, before this can be done, it is necessary to evaluate the stress field and update the effective viscosity. 

Our [definition](#Symmetric-and-resistive-gradient-operators) of the operator $\boldsymbol{\nabla}_R$ was prompted by the observation that $\mathbf{R}:= \mu\boldsymbol{\nabla}_R\mathbf{u}$ represents a list of the resistive stresses. Resistive stresses, in turn, can be used to calculate the deviatoric stresses via Equation 20; once deviatoric stresses are known, the effective stress is obtained via Equation 22, and, subsequently, the effective viscosity can be updated via Equation 21. 

In [22]:
#calculate the resistive stress vector:
R = μ*nabla_R(u) #[R_xx, R_yy, (R_zz), R_xy, R_xz, R_yz]^T

#And the associated deviatoric stresses (Eq. 20)
τ = sp.zeros(3+dim, 1)
τ[0] = sp.Rational(2,3)*R[0] - sp.Rational(1,3)*R[1] #τ_xx
τ[1] = sp.Rational(2,3)*R[1] - sp.Rational(1,3)*R[0] #τ_yy
τ[-3] = R[-3] #τ_xy
τ[-2] = R[-2] #τ_xz
τ[-1] = R[-1] #τ_yz
if hydrostatic_approximation: #include τ_zz, which would be omitted in BP:
    τ[2] = R[2] - sp.Rational(1,3)*(R[0] + R[1]) #τ_zz

#calculate the effective stress τ_E (Eq. 22)
expr = sum([τ[i]**2 for i in range(len(τ))])
if not hydrostatic_approximation: #if solving BP:
    τ_zz = -τ[0] - τ[1] #recover τ_zz by incompressibility
    expr += (τ_zz)**2 #and include this value in computing τ_E:
τ_E = sp.sqrt(sp.Rational(1,2)*expr)

#update the effective viscosity (Eq. 21)
μ_2 = 1/(2*A*τ_E**(n-1))

print('For example, the longitudinal deviatoric tension, τ_xx, is')
τ[0]

For example, the longitudinal deviatoric tension, τ_xx, is


g*h*ρ/6

## Nonlinear rheology solution

In general, to compute the nonlinear rheology solution, it would be necessary to run the whole scrip again with ```μ_2``` in place of ```μ```. But in this case, ```μ_2``` is just a function of $\rho$, $g$, $h$, and $A$, all of which were assumed constant:

In [23]:
print('Updated effective viscosity:')
μ_2 #compare with the biaxial extension value given in Eq. 40

Updated effective viscosity:


(sqrt(3)*g*h*ρ/6)**(1 - n)/(2*A)

Therefore, ```μ_2``` is a constant, like ```μ```, and a second iteration would just return the same solution with ```μ_2``` in place of ```μ```. In this case, we may as well update the solution manually:

### Nonlinear nodal solution

In [24]:
d_2 = sp.simplify(d*μ/μ_2) #cancel out μ and replace with μ_2 in the denominator

print('Nodal velocity solution (nonlinear rheology)')
print('Components are in their original order,', nodal_velocity[:4], '...', nodal_velocity[-4:], ':')
d_2.T

Nodal velocity solution (nonlinear rheology)
Components are in their original order, ['u_1x', 'u_1y', 'u_1z', 'u_2x'] ... ['u_7z', 'u_8x', 'u_8y', 'u_8z'] :


Matrix([[0, 0, 0, 0, 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 0, 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 0, 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 0, 0, 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 0, -2*6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 0, 0, -2*6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 0, 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), -2*6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), 6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1), -2*6**(-n)*A*g*h**2*ρ*(sqrt(3)*g*h*ρ)**(n - 1)]])

### Nonlinear interpolated solution

In [25]:
u_2 = sp.simplify(N*d_2) #interpolate via u_2 = N*d_2

if not symbolic_solution: #if solving numerically:
    u_2 = round_expr(u_2) #remove rounding-error-sized coefficients

print('Interpolated velocity solution (nonlinear rheology) for', velocity, ':')
u_2 #compare with the biaxial extension solution of Equation 41 from the paper

Interpolated velocity solution (nonlinear rheology) for ['u_x', 'u_y', 'u_z'] :


Matrix([
[   3**(n/2 - 1/2)*A*x*(g*h*ρ/6)**n],
[   3**(n/2 - 1/2)*A*y*(g*h*ρ/6)**n],
[-2*3**(n/2 - 1/2)*A*z*(g*h*ρ/6)**n]])

In [26]:
#print('runtime:', time.time() - start, 'seconds')